In [1]:
# cBioPortal_GeneExprMutation.v001
# Alberto Bejarano (2024)
import time; start = time.time()
from datetime import datetime; print(datetime.now().strftime("%B %d, %Y %H:%M:%S"))

June 07, 2024 21:19:03


In [2]:
# https://pybioportal.readthedocs.io/en/latest/index.html
# https://github.com/Matteo-Valerio/pyBioPortal/tree/master/examples
# https://github.com/LLCampos/pybioportal
# https://pypi.org/project/pybioportal/
# https://www.biostars.org/p/106127/
# https://docs.cbioportal.org/user-guide/faq/#what-are-mrna-and-microrna-z-scores
# https://www.ebi.ac.uk/gxa/FAQ.html

In [3]:
#!pip install pybioportal
#!conda install -c matteo.valerio pybioportal
#!pip install autograd

In [4]:
#%config InlineBackend.figure_format = "retina"
#%matplotlib inline

In [5]:
from pybioportal import server_running_status as srs
from pybioportal import molecular_profiles as mf
from pybioportal import molecular_data as md
from pybioportal import mutations as mts
from pybioportal import samples as sp
#          -          -          -          -          -          -          -          -          -          -          -  
import pandas as pd
pd.set_option('display.precision', 2)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.precision', 3)

#        -         -        -        -         -        -         -        -        -         -
import warnings; warnings.filterwarnings("ignore")
#from scipy.stats import mannwhitneyu, normaltest
#from statannotations.Annotator import Annotator
#from statannot import add_stat_annotation
import matplotlib.pyplot as plt
import matplotlib.transforms
import autograd.numpy as np
import seaborn as sns

In [6]:
srs.get_server_status()

,status
0,UP


In [7]:
GenesOfInterest = ["PDCD1", "CD274", "CTLA4"]
GenesOfInterest = ["5133", "29126", "1493"]
GenesOfInterest = ["PDCD1", "CD274", "CTLA4"]
GenesOfInterest = ['EGFR', 'ERBB2', 'PTEN', 'CCND1', 'TP53', 'CDKN1B', 'KRAS', 'NRAS', 'HRAS', 'BRAF', 'PIK3CA', 'TTN']

GenesOfInterest = ['TP53', 'TTN']

In [8]:
genes_df = pd.read_csv("./metadata/HGNC_table.txt", header=0, sep='\t', quotechar='"', on_bad_lines="warn")
genes_df = genes_df.dropna(subset=['NCBI Gene ID'])
genes_df["NCBI Gene ID"] = genes_df["NCBI Gene ID"].astype(int)
genes_df = genes_df[genes_df["Approved symbol"].isin(GenesOfInterest)]
genes_df = genes_df[['Approved symbol', 'NCBI Gene ID']].copy()
genes_df.rename({'NCBI Gene ID':'entrezGeneId', 'Approved symbol':'ApprovedSymbol'},axis=1,inplace=True)
genes_df.sample()

,ApprovedSymbol,entrezGeneId
45970,TTN,7273


In [9]:
genes = sorted(genes_df['ApprovedSymbol'].astype(str).unique().tolist()); print(genes)

['TP53', 'TTN']


In [10]:
NCBIgenes = sorted(genes_df['entrezGeneId'].astype(str).unique().tolist()); print(NCBIgenes)

['7157', '7273']


In [11]:
molecular_profiles01 = df_exp.molecularProfileId.unique().tolist()
print(len(molecular_profiles01)); print('\033[33m', sorted(molecular_profiles01), '\033[0m')

NameError: name 'df_exp' is not defined

In [ ]:
molecular_profiles02 = df_mut.molecularProfileId.unique().tolist()
print(len(molecular_profiles02)); print('\033[33m', sorted(molecular_profiles02), '\033[0m')

In [ ]:
#molecular_profiles02 = ['acc_tcga_mutations']

In [ ]:
%%time
exp_data = md.fetch_molecular_data(entrez_gene_ids=NCBIgenes, molecular_profile_ids=molecular_profiles01)
exp_data = exp_data[['sampleId', 'entrezGeneId', 'studyId', 'value']].copy()
exp_data.sample(3)

In [ ]:
%%time
mut_data = mts.fetch_muts_in_multiple_mol_profs(entrez_gene_ids=NCBIgenes, molecular_profile_ids=molecular_profiles02)
mut_data = mut_data[['sampleId', 'entrezGeneId', 'studyId', 'mutationType']].copy()
mut_data.sample(5)

In [ ]:
#mutationStatuss = mut_data.mutationStatus.unique().tolist(); print('\033[33m', sorted(mutationStatuss), '\033[0m')

In [ ]:
data = pd.merge(exp_data, mut_data, on=['studyId', 'sampleId', 'entrezGeneId'], how='left')
data = pd.merge(data, genes_df, on='entrezGeneId', how='inner')

In [ ]:
sampl_data = sp.fetch_samples(sample_list_ids=molecular_profiles01)
sampl_data = sampl_data[['sampleId', 'sampleType']].copy()
sampl_data.sample()

In [ ]:
data = pd.merge(data, sampl_data, on='sampleId', how='left')
data = data[['sampleId', 'studyId', 'sampleType', 'ApprovedSymbol', 'mutationType', 'value']].copy()
data['studyId'] = data['studyId'].str.replace('_tcga', '')

In [ ]:
data.fillna('No_Mutation', inplace=True)
mutationTypes = data.mutationType.unique().tolist(); print('\033[33m', mutationTypes, '\033[0m')

In [ ]:
data.mutationType.value_counts()

In [ ]:
data.sampleType.value_counts()

In [ ]:
impact_mapping = {
    'No_Mutation':            'No_impact',
    
    'Translation_Start_Site': 'Severe',  
    'Nonsense_Mutation':      'Severe',
    'Frame_Shift_Del':        'Severe',
    'Frame_Shift_Ins':        'Severe',
    
    'Missense_Mutation':      'Moderate',
    'Splice_Region':          'Moderate',
    'Splice_Site':            'Moderate',
    
    'In_Frame_Del':           'Mild',
    'In_Frame_Ins':           'Mild',

}

data['impact'] = data['mutationType'].map(impact_mapping)

In [ ]:
data_log2 = data.apply(lambda x: np.log2(x + 1) if np.issubdtype(x.dtype, np.number)else x)

In [ ]:
data_log2.sample(10)

In [ ]:
hue_order = mutationTypes

In [ ]:
genes = data_log2.ApprovedSymbol.unique().tolist(); print(genes)

In [ ]:
mutationType_palette = {'No_Mutation':'#00FF00',
                        'Frame_Shift_Del':'#e76f51',    'Frame_Shift_Ins':'#f4a261',
                        'In_Frame_Del':'#e76f51',       'In_Frame_Ins':'#e76f51',
                        'Missense_Mutation':'#06d6a0',
                        'Nonsense_Mutation':'#06d6a0', 
                        'Nonstop_Mutation':'#598392', 
                        'Splice_Site':'#598392',        'Splice_Region':'#598392',
                         'Translation_Start_Site':'#e76f51'
                       }

In [ ]:
Impact_palette = {'No_impact': '#7E7E7E',
                   'Severe':    '#FF0000',
                   'Moderate':  '#598392',
                   'Mild':      '#598392',
                       }

In [ ]:
for gene in sorted(genes):
    fig, ax  = plt.subplots(figsize=(25/2.54, 10/2.54), dpi=150)
    df02 = data_log2.loc[data_log2.ApprovedSymbol == gene]; n = len(df02)
    df02 = df02.dropna(axis=0, how='any')
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          -       
    median_df  = df02.groupby(['studyId'])['value'].median().sort_values(ascending=False).reset_index()
    indication_list = median_df.studyId.unique().tolist(); # print(indication_list)
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          -
    data = df02; x='studyId'; y='value'; hue="impact"; order=indication_list; palette=mutationType_palette
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          - 
    str_kws = {'s':5, 'jitter':.45, 'alpha':0.5, 'dodge':True, 'linewidth':0.2,'edgecolor':'#1E1E1E', 'palette': palette,
               'order':indication_list, 'hue_order':hue_order}
    box_kws = {'linewidth':0.5, 'width':0.75, 'dodge':True, 'showcaps':True, 'whis':1.5, 'fliersize':0, 'saturation':0.20, 
               'order':indication_list, 'hue_order':hue_order}
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          -  
    medianprops=dict(linestyle='-', linewidth=1, color="firebrick", alpha=0.5); capprops=dict(linestyle="-", alpha=.5, linewidth=.3);
    boxprops=dict(linestyle="-", alpha=.3, linewidth=.3); whiskerprops=dict(linestyle="--", linewidth=.3, color="#7E7E7E");
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          -    
    g = sns.stripplot(data=data, x=x, y=y, hue=hue, **str_kws, zorder=0, ax=ax)
    g = sns.boxplot(data=data,   x=x, y=y, hue=hue, medianprops=medianprops, 
                    capprops=capprops, boxprops=boxprops, whiskerprops=whiskerprops, **box_kws, zorder=1, ax=ax)
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          -   
    ax.set_title(gene, fontsize=14, color="#7E7E7E", pad=10)
    plt.xlabel("", fontsize=0); plt.ylabel("Gene Expression log2[TPM+1]", fontsize=6, color="#7E7E7E", labelpad=5)
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          -    
    ax.set_ylim(-.9, 15); ax.set_yscale('linear'); sns.despine(left=True, bottom=True)   
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          -        
    ax.tick_params(axis="x", colors="#cfcfcf", pad=0, length=2); ax.tick_params(axis="y", colors="#cfcfcf", pad=0, length=3)
    plt.xticks(fontsize=10, color="#7E7E7E", rotation=60, ha="center"); ax.tick_params(axis='x', which='major', pad=5)
    plt.yticks(fontsize=10, color="#7E7E7E", rotation=0, ha="center"); ax.tick_params(axis='y', which='major', pad=8)
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          -  
    kws_lines = {"ls":"--", "alpha":0.3, "linewidth":.5}
    plt.axhline(y=3.46, color="g", **kws_lines); plt.axhline(y=6.67, color="orange", **kws_lines); plt.axhline(y=9.97, color="red", **kws_lines)
    kws_box   = {'lw':.2, 'alpha':0.04, 'edgecolor':None}
    plt.axhspan(-1, 3.46, color='green', **kws_box); plt.axhspan(3.46, 6.67, color='orange', **kws_box); plt.axhspan(6.67, 15, color='red', **kws_box)
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          -       
    #annot_kws1 = {'pairs':pairs, 'data':data, 'x':x, 'y':y, 'order':order, 'hue':hue, 'hue_order':hue_order}
    #annot_params = {"test":"Mann-Whitney", "line_width":0.5, "color":"#7E7E7E", "text_format":"star", 'loc':'inside'}
    #try:
    #    ax, test_results = Annotator(ax, **annot_kws1).reset_configuration().configure(**annot_params, verbose=0).apply_and_annotate()
    #except Exception as inst:
    #    print(inst) 
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          -   
    handles, labels = ax.get_legend_handles_labels()
    plt.legend(handles[0:2],labels[0:2],fontsize=6,bbox_to_anchor=(1.02,1),loc=2,borderaxespad=0.1)
    plt.legend([],[],frameon=False)
    #          -          -          -          -          -          -          -          -          -          -          -          -          -          -       
    plt.tight_layout(); plt.show(); #plt.close()
    #plt.savefig("tcga.png", format="png", bbox_inches="tight", dpi=300)

In [ ]:
print("'cBioPortal_TCGA_MutationLandscape_v01' script run time:", f'{time.time()-start:.0f}', "seconds.")

# 